In [1]:
from imdynamo import ItemModel
import pandas as pd
import json

list = []

for i in ItemModel.scan(ItemModel.project_name=='amz_us'):
    object = json.loads(i.info)
    #print(object)
    list.append([object['asin'], object['sku'], object['product_name']])

df = pd.DataFrame(data = list, columns=['asin','sku','product_name'])
print(df)

          asin            sku                product_name
0   B082MDWMZY  51EF0850AA000     Creative Outlier ONE V2
1   B0777NPDK2  51EF0750AA000                Aurvana Trio
2   B0044DEDC0  70SB129000000  Sound Blaster X-Fi Go! Pro
3   B07BQVWHV3  70SB172000001           Sound Blaster K3+
4   B0791H74NT  51MF1680AA000             Creative Pebble
..         ...            ...                         ...
86  B07L83RMPJ  70GH040000000         Creative SXFI AIR C
87  B002EL4MXY  51MZ0085AA030                      EP-630
88  B00SA1IAWG  70GH030000000           Sound Blaster JAM
89  B07JC6YZ4K  51MF8360AA002              Creative Stage
90  B081ZN9K2M  70SB183000000            Sound Blaster G3

[91 rows x 3 columns]


In [2]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
import gzip
import csv
import s3fs

ModuleNotFoundError: No module named 'pyarrow'

In [224]:
fs = s3fs.S3FileSystem(anon=False)
s3 = boto3.resource('s3')

def p2f(x):
    return x.str.replace('—','0.00').str.strip('%')

def reformat(currentFolder, result):
    columns = [t.lower().replace(' ', '_').replace('#', '') for t in result.columns]
    result.columns = columns

    if currentFolder == 'ItemComparison':
        result['1_compared_%'] = (p2f(result['1_compared_%'])).astype(float)
        result['2_compared_%'] = (p2f(result['2_compared_%'])).astype(float)
        result['3_compared_%'] = (p2f(result['3_compared_%'])).astype(float)
        result['4_compared_%'] = (p2f(result['4_compared_%'])).astype(float)
        result['5_compared_%'] = (p2f(result['5_compared_%'])).astype(float)

    
    elif currentFolder == 'AlternatePurchase':
        result['1_purchased_%'] = (p2f(result['1_purchased_%'])).astype(float)
        result['2_purchased_%'] = (p2f(result['2_purchased_%'])).astype(float)
        result['3_purchased_%'] = (p2f(result['3_purchased_%'])).astype(float)
        result['4_purchased_%'] = (p2f(result['4_purchased_%'])).astype(float)
        result['5_purchased_%'] = (p2f(result['5_purchased_%'])).astype(float)

    elif currentFolder == 'BasketAnalysis':
        result['1_combination_%'] = (p2f(result['1_combination_%'])).astype(float)
        result['2_combination_%'] = (p2f(result['2_combination_%'])).astype(float)
        result['3_combination_%'] = (p2f(result['3_combination_%'])).astype(float)

    return result

In [233]:
fileCount = 0

currentFolder = 'BasketAnalysis'

my_bucket = s3.Bucket('im-data-lake-collected')
for data in my_bucket.objects.filter(Prefix='amazon/table='+ currentFolder + '/country=us/'):
    print(data.key)
    if '.gz' in data.key:
        with fs.open('s3://im-data-lake-collected/' + data.key) as f:
            df2 = pd.read_csv(f, header=1, compression='gzip')
            #print(df2)

        result = pd.merge(df2,df,left_on='ASIN', right_on='asin').drop('asin', axis=1)
        
        result = reformat(currentFolder, result)

        for col in result.columns:
            print(col)
        
        #print(result.dtypes)

        result.to_parquet('s3://im-data-lake/' + data.key.replace(".gz", ".parquet"), compression='gzip')
        fileCount+=1

amazon/table=BasketAnalysis/country=us/
amazon/table=BasketAnalysis/country=us/yy=2020/mm=11/dd=14/2020-11-14.gz
asin
product_title
1_purchased_asin
1_purchased_title
1_combination_%
2_purchased_asin
2_purchased_title
2_combination_%
3_purchased_asin
3_purchased_title
3_combination_%
sku
product_name
amazon/table=BasketAnalysis/country=us/yy=2020/mm=11/dd=21/2020-11-21.gz
amazon/table=BasketAnalysis/country=us/yy=2020/mm=11/dd=28/2020-11-28.gz
amazon/table=BasketAnalysis/country=us/yy=2020/mm=11/dd=7/2020-11-07.gz
amazon/table=BasketAnalysis/country=us/yy=2020/mm=12/dd=12/2020-12-12.gz
amazon/table=BasketAnalysis/country=us/yy=2020/mm=12/dd=19/2020-12-19.gz
amazon/table=BasketAnalysis/country=us/yy=2020/mm=12/dd=26/2020-12-26.gz
amazon/table=BasketAnalysis/country=us/yy=2020/mm=12/dd=5/2020-12-05.gz
amazon/table=BasketAnalysis/country=us/yy=2021/mm=1/dd=16/2021-01-16.gz
amazon/table=BasketAnalysis/country=us/yy=2021/mm=1/dd=2/2021-01-02.gz
amazon/table=BasketAnalysis/country=us/yy=2021

In [5]:
table = pq.read_table("../2020-11-14.parquet")
df=table.to_pandas()
print(df)

          ASIN                                      Product Title  \
0   B0791H74NT  Creative Pebble 2.0 USB-Powered Desktop Speake...   
1   B0791J2658  Creative Pebble 2.0 USB-Powered Desktop Speake...   
2   B07NWLWM9B  Creative Pebble Plus 2.1 USB-Powered Desktop S...   
3   B07VVP8BGD  Creative Pebble V2 - Minimalistic 2.0 USB-C Po...   
4   B08F57GSJ7  Creative Pebble V3 Minimalistic 2.0 USB-C Desk...   
5   B00EO6X4XG  Creative Sound Blaster Audigy FX PCIe 5.1 Soun...   
6   B07FY45F2S  Sound BlasterX G6 Hi-Res 130dB 32bit/384kHz Ga...   
7   B06XBZ38ZJ  Creative Labs Sound Blaster Play! 3 External U...   
8   B07JC6YZ4K  Creative Stage 2.1 Channel Under-Monitor Sound...   
9   B001RNOHDU  Creative Labs 51MF1610AA002 GigaWorks T20 Seri...   
10  B07HXCLM54  Creative Stage Air Portable and Compact Under-...   
11  B07MPZB77S  Creative Outlier Air TWS True Wireless Sweatpr...   
12  B07MPZB77S  Creative Outlier Air TWS True Wireless Sweatpr...   
13  B07BQVWHV3  Creative Sound Bla

In [6]:
table = pq.read_table("../2020-11-14.parquet")
df=table.to_pandas()
print(df)

          ASIN                                      Product Title  \
0   B0791H74NT  Creative Pebble 2.0 USB-Powered Desktop Speake...   
1   B0791J2658  Creative Pebble 2.0 USB-Powered Desktop Speake...   
2   B07NWLWM9B  Creative Pebble Plus 2.1 USB-Powered Desktop S...   
3   B07VVP8BGD  Creative Pebble V2 - Minimalistic 2.0 USB-C Po...   
4   B08F57GSJ7  Creative Pebble V3 Minimalistic 2.0 USB-C Desk...   
5   B00EO6X4XG  Creative Sound Blaster Audigy FX PCIe 5.1 Soun...   
6   B07FY45F2S  Sound BlasterX G6 Hi-Res 130dB 32bit/384kHz Ga...   
7   B06XBZ38ZJ  Creative Labs Sound Blaster Play! 3 External U...   
8   B07JC6YZ4K  Creative Stage 2.1 Channel Under-Monitor Sound...   
9   B001RNOHDU  Creative Labs 51MF1610AA002 GigaWorks T20 Seri...   
10  B07HXCLM54  Creative Stage Air Portable and Compact Under-...   
11  B07MPZB77S  Creative Outlier Air TWS True Wireless Sweatpr...   
12  B07MPZB77S  Creative Outlier Air TWS True Wireless Sweatpr...   
13  B07BQVWHV3  Creative Sound Bla